# USPS Problem

In [ ]:
# import the csv data files using pandas
import pandas as pd

# create DataFrame for the deliveries
df_deliveries = pd.read_csv("deliveries.csv").set_index('Delivery')

# convert the DataFrame into dictionaries (for convenience--it is also possible to directly use df_deliveries)
Origin = {}
Destination = {}
Available = {}
LatestArrival = {}
Volume = {}
for i in df_deliveries.index:
    Origin[i] = df_deliveries['Origin'][i]  # this assigns the i-th element
    Destination[i] = df_deliveries['Destination'][i]
    Available[i] = df_deliveries['Available'][i]
    LatestArrival[i] = df_deliveries['LatestArrival'][i]
    Volume[i] = df_deliveries['Volume'][i]

In [ ]:
# Create DataFrame for the trips
df_trips = pd.read_csv("trips.csv").set_index('Trip')

# Convert the DataFrame into dictionaries (for convenience--it is also possible to directly use df_trips).
# The following parameters are easy to define:
Cost = {}
Capacity = {}
Leave = {}
Arrive = {}
for i in df_trips.index:
    Cost[i] = df_trips['Cost'][i]
    Capacity[i] = df_trips['Capacity'][i]
    Leave[i] = df_trips['Leave'][i]
    Arrive[i] = df_trips['Arrive'][i]

In [ ]:
# visits are a bit more challenging: I want this to be a set (or a list) for each trip
Visits = {}
for i in df_trips.index:
    tmpList = []
    # each trip has at least two visits
    tmpList.append(df_trips['Visit 1'][i])
    tmpList.append(df_trips['Visit 2'][i])
    # and optionally a third visit (check if not not null)
    if pd.notnull(df_trips['Visit 3'][i]):
        # note: because of the missing values (NaN), this column is interpreted as type "float".
        # to make it an integer element, I cast it as "int"        
        tmpList.append(int(df_trips['Visit 3'][i]))
    Visits[i] = tmpList

In [ ]:
# for convenience, define two more index sets
Deliveries = df_deliveries.index
Trips = df_trips.index

In [ ]:
# Define delivery/trip compatibility
#
# We are making some strong assumptions here: 
# - all deliveries are from 1 to 4, from 1 to 5, from 2 to 4, or from 2 to 5;
# - trips are either direct or via hub denoted by 3 (all starting from {1,2} and ending in {4,5})
# Therefore, we can simply test whether the origin and destination are part of the visits of each trip, and
# whether the Available/Leave times and LatestArrival/Arrive times are compatible.
#
# In general, we would need to have access to the direction of the trip, as well as the intermediate
# times of the visits to compile this informatio (as is done in the USPS paper).
CanAssign = {}
for i in Deliveries:
    for r in Trips:
        # (note: we can use a backslash \ to continue a statement on the next line)
        if Origin[i] in Visits[r] and Destination[i] in Visits[r] \
        and Available[i] <= Leave[r] and Arrive[r] <= LatestArrival[i]:
            CanAssign[i,r] = 1